In [74]:
from ase import Atoms
from ase.build import bulk
from ase.io import write

# Make the Argon FCC Lattice

In [75]:
lattice_constant = 5.26

In [76]:
# Bulk argon structure
atoms = bulk('Ar', 'fcc', a=lattice_constant, cubic=True)

supercell = atoms.repeat((8, 8, 8))

write('Ar.data', supercell, format='lammps-data')

In [77]:
# Define Simulation Parameters

In [78]:
# Define Variables
N = 256
kb = 1.380649e-23

# Conversions
atmToPascal = 101325
angstrom3Tom3 = 1e-30

# Calculation Variables
totalSteps = 100000
framesPerSave = 90

RDF_Bins = 50

PVNRT_sim_data = "slurm-528.out"
RDF_sim_data = "RDF_Sim5.data"


#PVNRT_Name_Sim2 = "slurm-525.out"
#RDF_Name_Sim2 = "RDF.data"

In [79]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List


# --- If you already have your own read_TPVE_File, keep it and skip this stub. ---
def read_TPVE_File(path: str, totalSteps: int, saveEveryNFrames: int) -> pd.DataFrame:
    cols = ["Step", "Temp", "Press", "Volume", "TotEng"]
    total_line_count = sum(1 for _ in open(path, "r"))
    pvnrt_lines = totalSteps // saveEveryNFrames
    skip_rows = max(0, total_line_count - pvnrt_lines - 28)
    df = pd.read_csv(path, sep=r"\s+", header=None, names=cols,
                     skiprows=skip_rows, nrows=pvnrt_lines + 1, engine="python")
    return df
# ----------

def load_rdf_file(filepath: str, bins: int) -> List[pd.DataFrame]:
    """
    Read an RDF output file from LAMMPS and return
    a list of DataFrames, one per RDF snapshot.
    """

    # Columns expected in each RDF block
    col_names = ["Bin", "r", "g_Ar-Ar", "coord_Ar-Ar"]

    file_path = Path(filepath)
    if not file_path.exists():
        raise FileNotFoundError(f"Could not find RDF file: {filepath}")

    # Count total lines without loading whole file into memory
    with file_path.open("r") as fh:
        total_lines = sum(1 for _ in fh)

    # Number of RDF datasets in file
    group_count = (total_lines - 5) // (bins + 1)

    rdf_list: List[pd.DataFrame] = []

    # Parse each RDF dataset
    for group_idx in range(group_count + 1):
        start_line = 4 + group_idx * (bins + 1)
        rdf_block = pd.read_csv(
            file_path,
            sep=r"\s+",
            header=None,
            names=col_names,
            skiprows=start_line,
            nrows=bins,
            engine="python"
        )
        if rdf_block.empty:
            continue
        rdf_list.append(rdf_block)

    return rdf_list

In [80]:
# Plotting Functions

In [81]:
import matplotlib.pyplot as plt
import pandas as pd

def show_rdf(single_df: pd.DataFrame, sim_info: str) -> None:
    """Plot RDF curve for one dataset."""
    plt.figure(figsize=(18, 14))
    plt.plot(single_df["r"], single_df["g_Ar-Ar"], label="Ar–Ar RDF")

    plt.xlabel("Radius (Å)")
    plt.ylabel("g(r)")
    plt.title(f"Radial Distribution Function for FCC Argon ({sim_info})")
    plt.legend()
    plt.grid(True, linestyle="--", linewidth=0.7)
    plt.show()


def compare_rdfs(list_of_dfs: list[pd.DataFrame], sim_info: str, labels: list[str]) -> None:
    """Overlay multiple RDF curves on the same plot."""
    plt.figure(figsize=(18, 14))
    for df, lbl in zip(list_of_dfs, labels):
        plt.plot(df["r"], df["g_Ar-Ar"], label=lbl)

    plt.xlabel("Radius (Å)")
    plt.ylabel("g(r)")
    plt.title(f"Comparison of RDFs for FCC Argon ({sim_info})")
    plt.legend()
    plt.grid(True, linestyle="--", linewidth=0.7)
    plt.show()


def plot_pv_minus_nkbt(thermo_df: pd.DataFrame, add_zero: bool = True) -> None:
    """Plot (PV – NkBT) vs simulation step."""
    plt.figure(figsize=(18, 14))

    if add_zero:
        plt.axhline(y=0.0, color="black", linestyle="--", linewidth=1)

    steps = thermo_df["Step"]
    T = thermo_df["Temp"]
    P = thermo_df["Press"] * atmToPascal
    V = thermo_df["Volume"] * angstrom3Tom3

    pv = P * V
    nkbt = N * kb * T
    deviation = pv - nkbt

    plt.plot(steps, deviation, label="Argon (PV – NkBT)")
    plt.xlabel("Simulation Step")
    plt.ylabel("Deviation from Ideal Gas Law")
    plt.title("PV – NkBT over NVT Argon FCC Simulation")
    plt.legend()
    plt.grid(True, linestyle="--", linewidth=0.7)
    plt.show()


# Plotting: PV - Nk_BT 